# FedLib: Implementation for FedAvg

## Importing supportive libaries
This notebook shows a demo on PyTorch back-end model impelementation.

In the very begining, we import the supporting libraries.

In [1]:
from fedlib.networks import resnet20

from fedlib.lib.algo import fedavg

from fedlib.ve import simulator
from fedlib.lib import Server
import fedlib

# Initialize Dataset 
Here we initialize Non-IID data for clients and test data for both clients and server.
You could use `fedlib.init_dataset`


In [2]:
data_args = {
'n_clients': 100,
'partition' : 'noniid-labeldir',
'dataset' : 'cifar10',
'datadir' : './data',
'beta' : .5,
'train_bs' : 64,
'test_bs' : 128,
'n_worker' : 32
}

data_loaders, global_test_dl, _ = fedlib.init_dataset(**data_args)



Files already downloaded and verified
Files already downloaded and verified


INFO:root:Data statistics: {0: {0: 20, 1: 24, 2: 40, 3: 69, 4: 98, 5: 10, 7: 108, 8: 68, 9: 39}, 1: {0: 80, 1: 111, 2: 95, 3: 3, 4: 121, 5: 17, 6: 1, 8: 185}, 2: {0: 3, 1: 84, 2: 14, 3: 69, 4: 4, 5: 51, 6: 18, 7: 1, 8: 145, 9: 28}, 3: {0: 169, 1: 47, 2: 38, 3: 2, 4: 42, 5: 2, 6: 1, 7: 50, 9: 8}, 4: {0: 45, 1: 32, 2: 10, 3: 4, 4: 12, 5: 109, 6: 9, 7: 6, 8: 19, 9: 12}, 5: {0: 138, 1: 4, 2: 15, 3: 25, 4: 15, 5: 112, 6: 1, 7: 8, 8: 49, 9: 33}, 6: {0: 5, 1: 1, 2: 84, 3: 31, 4: 61, 5: 2, 6: 6, 7: 19, 8: 74, 9: 13}, 7: {0: 83, 1: 10, 2: 8, 3: 5, 4: 48, 5: 81, 6: 9, 7: 18, 8: 6, 9: 22}, 8: {0: 1, 1: 1, 2: 41, 3: 39, 4: 74, 5: 18, 6: 83, 7: 1, 8: 63, 9: 38}, 9: {0: 142, 1: 23, 2: 19, 3: 91, 4: 2, 5: 178, 6: 27, 7: 3, 8: 86}, 10: {0: 73, 1: 11, 2: 30, 3: 7, 5: 39, 6: 3, 7: 2, 8: 17, 9: 2}, 11: {0: 8, 2: 93, 3: 37, 4: 12, 5: 19, 6: 7, 7: 128, 8: 13}, 12: {0: 4, 1: 9, 2: 70, 3: 8, 4: 1, 5: 27, 7: 90, 8: 10, 9: 5}, 13: {0: 3, 1: 1, 2: 112, 3: 129, 4: 155, 5: 15, 6: 25, 7: 344}, 14: {0: 131, 1: 45, 

Files already downloaded and verified
Files already downloaded and verified


INFO:root:Client ID:1,	Local Train Data Size:50000
INFO:root:Client ID:2,	Local Train Data Size:50000
INFO:root:Client ID:3,	Local Train Data Size:50000
INFO:root:Client ID:4,	Local Train Data Size:50000
INFO:root:Client ID:5,	Local Train Data Size:50000
INFO:root:Client ID:6,	Local Train Data Size:50000
INFO:root:Client ID:7,	Local Train Data Size:50000
INFO:root:Client ID:8,	Local Train Data Size:50000
INFO:root:Client ID:9,	Local Train Data Size:50000
INFO:root:Client ID:10,	Local Train Data Size:50000
INFO:root:Client ID:11,	Local Train Data Size:50000
INFO:root:Client ID:12,	Local Train Data Size:50000
INFO:root:Client ID:13,	Local Train Data Size:50000
INFO:root:Client ID:14,	Local Train Data Size:50000
INFO:root:Client ID:15,	Local Train Data Size:50000
INFO:root:Client ID:16,	Local Train Data Size:50000
INFO:root:Client ID:17,	Local Train Data Size:50000
INFO:root:Client ID:18,	Local Train Data Size:50000
INFO:root:Client ID:19,	Local Train Data Size:50000
INFO:root:Client ID:2

# Define the Model you want to deploy to FL
Here I use the ResNet-20 as an example.

In [3]:
model = resnet20()


# Define the FL algorithm
Here I use the fedavg imported from fedlib, you could import other algorithms or you could also write your own algorithm

In [4]:
trainer = fedavg(fedlib.get_logger())

## Create server and clients objects
Here we use the arguments we defined before, and create server and clients.

In [5]:
server_args = {
'n_clients' : 100,
'global_model' : model,
'device' : 'cpu', 
'sampler' : 'random',
'test_dataset' : global_test_dl,
'trainer' : trainer,
'communicator' : None
}
server = Server(**server_args)

In [6]:
client_args ={
    'n_clients': 100,
    'model': model,
    'data_loaders':data_loaders,
    'testloader':global_test_dl,
    'trainer':trainer, 
    'lr':0.01,
    'lr_scheduler':'ExponentialLR',
    'criterion':'CrossEntropyLoss',
    'optimizer':'sgd',
    'device': 'cpu'
}

clients = fedlib.init_clients(**client_args)

trainer: <fedlib.lib.algo.fedavg.fedavg.Trainer object at 0x7f85e2d46da0>


## Create simulator

Simulator simulates the virtual federated learning environments, and run server and clients on single device.

In [7]:
simulator = simulator(server=server, clients=clients, 
                        communication_rounds=10,
                        n_clients=100,
                        participate_rate=0.1)

## Run simulator
User API Simulator.run

In [8]:
simulator.run(local_epochs=2)

INFO:root:*******starting Rounds 1 Optimization******
INFO:root:Participate Clients: [ 2 61 89  5 70 39 31 34 27 90]
INFO:root:Optimize the 2-th Clients
INFO:root:Epoch: 0	Loss: 3.367300	Accuracy:0.081597%
INFO:root:Epoch: 1	Loss: 2.553155	Accuracy:0.366319%
INFO:root:*******Client 2 Training Finished! Test Accuracy: 0.1862 ******
INFO:root:Optimize the 61-th Clients
INFO:root:Epoch: 0	Loss: 3.243901	Accuracy:0.085938%
INFO:root:Epoch: 1	Loss: 2.527221	Accuracy:0.505208%
INFO:root:*******Client 61 Training Finished! Test Accuracy: 0.1087 ******
INFO:root:Optimize the 89-th Clients
INFO:root:Epoch: 0	Loss: 1.888654	Accuracy:0.578125%
INFO:root:Epoch: 1	Loss: 1.330540	Accuracy:0.847356%
INFO:root:*******Client 89 Training Finished! Test Accuracy: 0.1071 ******
INFO:root:Optimize the 5-th Clients
INFO:root:Epoch: 0	Loss: 4.177335	Accuracy:0.054688%
INFO:root:Epoch: 1	Loss: 3.367901	Accuracy:0.222656%
